In [16]:
import re
import json

In [17]:
scores_map = {}

def add_word(word, score, key):
    global scores_map
    if word in scores_map:
        scores_map[word][key] = score
    else:
        scores_map[word] = {key:score}

In [18]:
scores = ["positive-words.txt"]
for fpath in scores:
    path = "./score_sets/"+fpath
    with open(path, 'r') as file:
        for line in file:
            add_word(line.lower().strip(), 3, "txt")

scores = ["negative-words.txt"]
for fpath in scores:
    path = "./score_sets/"+fpath
    with open(path, 'r') as file:
        for line in file:
            add_word(line.lower().strip(), -3, "txt")

In [19]:
scores = ["SWN.txt"]


for fpath in scores:
    # pattern = r"word1=(.*?)\spos1"
    # pattern2 = r"priorpolarity=(.*?)\n"
    path = "./score_sets/"+fpath
    with open(path, 'r') as file:
        for line in file:
            fields = line.strip().split("\t")
            # POS	ID	PosScore	NegScore	SynsetTerms	Gloss
            score = 1 - (float(fields[2]) + float(fields[3]))
            for key in fields[4].split():
                wordd = key.split("#")[0].lower()
                add_word(wordd, score, "swn")

In [20]:
scores = ["subjcluesl.txt"]
for fpath in scores:
    pattern = r"word1=(.*?)\spos1"
    pattern2 = r"priorpolarity=(.*?)\n"
    path = "./score_sets/"+fpath
    with open(path, 'r') as file:
        for line in file:
            match2 = re.search(pattern2, line)
            match = re.search(pattern, line)
            word = match.group(1)
            word2 = match2.group(1)[:3]
            if(word2 == "pos"):
                add_word(word.lower(),3, "sub")
            elif(word2 == "neg"):
                add_word(word.lower(),-3, "sub")
            elif(word2 == "neu"):
                add_word(word.lower(),0, "sub")

In [21]:

with open("./score_sets/AFINN.txt", "r") as f:
    for line in f:
        fields = line.strip().split("\t")
        add_word(fields[0].lower(),fields[1], "affin")
        word = fields[0]

In [101]:
def write_dictionary(dictionary, file_path):
    with open(file_path, 'w') as file:
        json.dump(dictionary, file)

def read_dictionary(file_path):
    with open(file_path, 'r') as file:
        dictionary = json.load(file)
    return dictionary

# write_dictionary(scores_map, "./sets/scores_map.json")
scores_map = read_dictionary("./sets/scores_map.json")
assert len(scores_map) == 149567

In [106]:

def count_word_occurrences(file_path):
    word_counts = {}
    with open(file_path, 'r') as file:
        for line in file:
            word = line.strip().lower()
            if word in word_counts:
                word_counts[word] += 1
            else:
                word_counts[word] = 1
    return word_counts

def find_common_keys(dict1, dict2):
    common_keys = []
    new_map = {}

    for key in dict2.keys():
        if key in dict1:
            common_keys.append(key)
        else:
            new_map[key] = dict2[key]
    return common_keys, new_map

In [121]:
file_pos = "./sets/pos_traits.txt"
file_neg = "./sets/neg_traits.txt"
file_neu = "./sets/neu_traits.txt"
files_new = ["./sets/traits.txt", "./sets/traits2.txt", "./sets/traits3.txt", "./sets/traits4.txt", "./sets/traits5.txt", "./sets/traits6.txt"]

pos_occ = count_word_occurrences(file_pos)
neg_occ = count_word_occurrences(file_neg)
neu_occ = count_word_occurrences(file_neu)
undone = []
print(len(pos_occ),len(neg_occ),len(neu_occ))

def find_label(sc):
    if("affin" in sc):
        lab = sc["affin"]
    elif("sub" in sc):
        lab = sc["sub"]
    elif("txt" in sc):
        lab = sc["txt"]
    elif("swn" in sc):
        lab = sc["swn"]

    return float(lab)

neu_last = {}
pos_last = {}
neg_last = {}

for file_new in files_new:
    new_occ = count_word_occurrences(file_new)

    _, new_map = find_common_keys(pos_occ, new_occ)
    _, new_map = find_common_keys(neg_occ, new_map)
    _, new_map = find_common_keys(neu_occ, new_map)

    for key in new_map:
        if key in scores_map:
            label = find_label(scores_map[key])

            if(label == 0):
                neu_occ[key] = 1
            if(label > 0):
                pos_occ[key] = 1
            if(label < 0):
                neg_occ[key] = 1
        else:
            undone.append(key)
print(len(pos_occ),len(neg_occ),len(neu_occ))


517 700 84
877 911 95


In [122]:
path = "./sets/unfinished.txt"
with open(path, 'r') as file:
    for line in file:
        fields = line.split(",")
        key = fields[0].strip()
        score = float(fields[1].strip())
        if(score == 0):
            if key in neu_occ:
                neu_occ[key] += 1
            else:    
                neu_occ[key] = 1
            if key in pos_occ:
                del pos_occ[key]
            if key in neg_occ:
                del neg_occ[key]
        elif(score > 0):
            if key in pos_occ:
                pos_occ[key] += 1
            else:    
                pos_occ[key] = 1
            if key in neu_occ:
                del neu_occ[key]
            if key in neg_occ:
                del neg_occ[key]
        elif(score < 0):
            if key in neg_occ:
                neg_occ[key] += 1
            else:    
                neg_occ[key] = 1
            if key in pos_occ:
                del pos_occ[key]
            if key in neu_occ:
                del neu_occ[key]

print(len(pos_occ),len(neg_occ),len(neu_occ))


827 1030 103


In [123]:
def find_repeated_keys(dict1, dict2):
    keys1 = set(dict1.keys())
    keys2 = set(dict2.keys())
    repeated_keys = keys1.intersection(keys2)
    return repeated_keys

# print(len(pos_occ),len(neg_occ),len(neu_occ))
out1 = find_repeated_keys(pos_occ, neg_occ)
for key in out1:
    print(pos_occ[key], neg_occ[key])
out2 = find_repeated_keys(pos_occ, neu_occ)
out3 = find_repeated_keys(neu_occ, neg_occ)
print(out1)
print(out2)
print(out3)
# print(pos_occ["meticulous"],neg_occ["meticulous"], neu_occ["meticulous"])


1 1
2 1
1 1
1 1
3 2
3 1
2 1
5 1
1 1
2 2
1 1
3 1
1 2
1 2
2 1
{'bold', 'protective', 'repentant', 'tough', 'assertive', 'determined', 'serious', 'precise', 'competitive', 'transparent', 'obedient', 'spontaneous', 'mannered', 'trendy', 'meticulous'}
{'thorough', 'protective', 'daring', 'observant', 'perceptive', 'subtle', 'relaxed', 'calm', 'simple', 'careful', 'scrupulous', 'intuitive', 'serious', 'discreet', 'playful', 'insightful', 'meticulous'}
{'ritualistic', 'protective', 'quiet', 'dogmatic', 'stiff', 'conformist', 'cautious', 'strange', 'secretive', 'quirky', 'opinionated', 'serious', 'picky', 'fanciful', 'unsophisticated', 'meticulous', 'stern'}


In [124]:
rem1 = ['bold', 'protective', 'repentant', 'tough', 'assertive', 'determined', 'serious', 'precise', 'transparent', 'obedient', 'spontaneous', 'mannered', 'trendy', 'meticulous']
rem2 = ['competitive']
for i in rem1:
    del neg_occ[i]

for i in rem2:
    del pos_occ[i]

In [125]:
rem1 = ['thorough', 'protective', 'daring', 'observant', 'perceptive', 'subtle', 'relaxed', 'calm', 'simple', 'careful', 'scrupulous', 'intuitive', 'serious', 'discreet', 'playful', 'insightful', 'meticulous']
for i in rem1:
    del neu_occ[i]


In [126]:
rem1 = ['ritualistic', 'quiet', 'dogmatic', 'stiff', 'conformist', 'strange', 'secretive', 'quirky', 'opinionated', 'unsophisticated', 'picky', 'fanciful', 'stern']
rem2 = ['cautious']
for i in rem1:
    del neu_occ[i]
for i in rem2:
    del neu_occ[i]
    del neg_occ[i]
pos_occ['cautious'] = 1

In [127]:
def find_repeated_keys(dict1, dict2):
    keys1 = set(dict1.keys())
    keys2 = set(dict2.keys())
    repeated_keys = keys1.intersection(keys2)
    return repeated_keys

# print(len(pos_occ),len(neg_occ),len(neu_occ))
out1 = find_repeated_keys(pos_occ, neg_occ)
out2 = find_repeated_keys(pos_occ, neu_occ)
out3 = find_repeated_keys(neu_occ, neg_occ)
print(out1)
print(out2)
print(out3)
# print(pos_occ["meticulous"],neg_occ["meticulous"], neu_occ["meticulous"])


set()
set()
set()


In [128]:

import csv

with open("sets/traits_scores.csv", 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Trait', 'Score'])  # Write header

    for key, value in pos_occ.items():
        writer.writerow([key, 1])
    for key, value in neg_occ.items():
        writer.writerow([key, -1])
    for key, value in neu_occ.items():
        writer.writerow([key, 0])

In [129]:
print(len(pos_occ),len(neg_occ),len(neu_occ))


827 1015 72
